In [ ]:
import os
import shutil

from dateutil.parser import parse
import django
import pandas as pd

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import *
# from mastspec.views import *
# from mastspec.forms import *
from utils import rows, columns, modeldict

In [ ]:
S_PATH = "/home/michael/Downloads/MSP_sample_data/"
BASE_DIR = os.getcwd()
IMAGE_PATH = os.path.join(BASE_DIR, "static_in_pro/our_static/img/")

In [ ]:
metaframe = pd.read_csv(S_PATH+'Metadata.csv')\
    .dropna()\
    .drop(columns=['Focal Distance.1'])\

metaframe.columns=['sol','mcam','name','rover_el','target_el','tau',
              'ltst','f_dist','i_angle','e_angle','so_lon','site',
              'drive','lat','lon','traverse']


In [ ]:
for obs in MObs.objects.all():
    obs.delete()

In [ ]:
for obs in rows(metaframe):
    obs = MObs(**dict(zip(obs.index,obs.values)))
    obs.ltst = parse(obs.ltst).time()
    if obs.target_el == '#NAME?':
        obs.target_el = None
    elif obs.target_el == 'Infinity':
        obs.target_el = float('inf')
    obs.save()

In [ ]:
MObs.objects.all()

In [ ]:
# hopefully this is consistent across the full data set!
# i think so; it's just sorted alphanumeric increasing
# without accounting for magnitude. if not, can make it more sophisticated

filter_index = [
    'L6', 'R6', 'L2', 'R2', 'R0_blue', 'L0_blue', 'L1', 
    'R1', 'R0_green', 'L0_green', 'R0_red', 'L0_red',
    'L4', 'L3', 'R3', 'L5', 'R4', 'R5'
]

In [ ]:
for spec in MSpec.objects.all():
    spec.delete()

In [ ]:
directories = [S_PATH+file for file in os.listdir(S_PATH) if os.path.isdir(S_PATH+file)]

for directory in directories:
    mcam = directory[directory.find('mcam'):]
    obs = MObs.objects.get(mcam__iexact=mcam)
    # sometimes the directory contains /working; sometimes /Working
    path = directory+'/'+os.listdir(directory)[0]+'/'
    contents = os.listdir(path) 
    spec_files = [file for file in contents if file[-3:] == 'csv']
    for spec_file in spec_files:
        frame = pd.read_csv(path+spec_file).T
        columns = filter_index + list(rows(frame)[0].values[18:])
        frame = frame.drop(['# Wavelength (nm)', ' Eye'])
        frame.columns = columns
        index = []
        for n in range(len(frame.index)):
            line = frame.iloc[n]
            # mean and stdev are on separate lines per filter.
            # some metadata is duplicated between these lines.
            if 'Mean' not in line.name:
                continue
            stdev_line = frame.iloc[n+1]
            color_end = line.name.find('Mean')
            color = line.name[1:color_end-1]
            spectrum = MSpec()
            if 'R2' in spec_file:
                spectrum.image_number = 2
            spectrum.observation = obs
            spectrum.roi_color = color
            for filt in filter_index:
                setattr(spectrum,filt+'_mean',float(line[filt]))
                setattr(spectrum,filt+'_stdev',float(stdev_line[filt]))
            if 'Float' in frame.columns:
                if line['Float']:
                    spectrum.is_floating = True
            for column_name in ['notes', 'Group', 'Formation', 'Member']:
                if column_name in frame.columns:
                    setattr(spectrum,column_name.lower(),line[column_name])
            spectrum.save()
    image_files = [file for file in contents if file[-3:] in ['png','jpg']] 
    for file in image_files:
        # this is _right_ on the edge of where i should be writing regexes.
        # assuming whatever 'convention' they have in the full data set
        # is even regexable.
        shutil.copy(path+file, IMAGE_PATH+file)
        if 'OVERLA' in file:
            if ('L126' in file) or ('L256' in file) or ('L20R3' in file):
                if '_R2.' in file:
                    obs.lefteye_s_image_2 = file
                else:
                    obs.lefteye_s_image_1 = file
            elif ('R126' in file) or ('R256' in file) or ('R20R3' in file):
                if '_R2.' in file:
                    obs.righteye_s_image_2 = file
                else:
                    obs.righteye_s_image_1 = file
        elif 'RGB':
            if 'L_' in file:
                obs.lefteye_rgb_image_1 = file
            elif 'R2' in file:
                obs.righteye_rgb_image_2 = file
            elif 'R' in file:
                obs.righteye_rgb_image_1 = file
    obs.save()